라이브러리 실행

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import modi
import time

from ai_camera import AICamera

bundle = modi.MODI(conn_mode="can")

print('bundle finish')


Running PyMODI (v1.0.1)
Initiating can connection...
Start initializing connected MODI modules
Motor (2606) has been connected!
Ir (620) has been connected!
Led (300) has been connected!
Speaker (1061) has been connected!
Ir (1862) has been connected!
Ir (1862) is not up to date. Please update the module by calling modi.update_module_firmware
Battery Module detected. Topology may by inaccurate.
Battery Module detected. Topology may by inaccurate.
 (1061) has been disconnected!!
 (2606) has been disconnected!!
MODI init timeout over. Check your module connection.
MODI modules are initialized!
bundle finish


유틸 함수

In [2]:
# 선 좌표 처리함수
def make_coordinates(image, line_parameters):
    slope, intercept = line_parameters
    y1 = image.shape[0]
    y2 = int(y1*(2/5))
    
    x1 = int((y1 - intercept)/slope)
    x2 = int((y2 - intercept)/slope)
    return np.array([x1, y1, x2, y2])

# 선 기울기 처리함수
def average_slope_intercept(image, lines):
    left_fit = []
    right_fit = []
    horizon_fit =[]
    for line in lines:
     
        x1, y1, x2, y2 = line.reshape(4)
        parameters = np.polyfit((x1, x2), (y1, y2), 1)
        
        slope = parameters[0]
        intercept = parameters[1]
        if slope < -0.5:
            left_fit.append((slope, intercept))
        elif 0.5 < slope:
            right_fit.append((slope, intercept))

    if (len(left_fit) != 0):
        left_fit_average = np.average(left_fit, axis=0)
    else:
        left_fit_average = ((-5, 240))
    if (len(right_fit) != 0):
        right_fit_average = np.average(right_fit, axis=0)
    else: 
        right_fit_average = ((5, -1360))    
    
    left_line = make_coordinates(image, left_fit_average)
    
    rigth_line = make_coordinates(image, right_fit_average)
    
    return np.array([left_line, rigth_line])

# Canny 이미지 처리함수
def canny(image):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    canny = cv2.Canny(blur, 50, 150)
    return canny

# 선 이미지 출력 함수
def t_display_lines(image, lines):
    line_image = np.zeros_like(image)
    
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line.reshape(4)
            
            cv2.line(line_image, (x1, y1), (x2, y2), (255, 0, 0), 10)
    return line_image

# Roi 설정 함수 (카메라 화각이 좁은 관계로 화면 전체를 잡은 상태)
def region_of_interest(image):
    height = image.shape[0]
    polygons = np.array([[( 0 , height), (320, height),(320,0) , (0, 0)]])
    mask = np.zeros_like(image)
    cv2.fillPoly(mask, polygons, 255)
    
    masked_image = cv2.bitwise_and(image, mask)
    return masked_image

# 중심 축과 검출된 선들의 중심축 좌표 출력 함수 
def find_vanishing(image, lines):
    
    x11, y11, x12, y12 = lines[0]
    cv2.line(image, (x11, y11), (x12, y12), (255, 255, 0), 10)
    x21, y21, x22, y22 = lines[1]
    cv2.line(image, (x21, y21), (x22, y22), (0, 255, 255), 3)
    m1 = (y12 - y11) / (x12 - x11)
    m2 = ((y22 - y21) / (x22 - x21)) -0.001
    cx = int((x11 * m1 - y11 - x21 * m2 + y21) / (m1 - m2))
    center = int(image.shape[1]/2)

    
    cv2.line(image, (cx, 0), (cx, image.shape[0]), (0, 0, 255), 3) 
    cv2.putText(image, str(cx), (cx+10, 100), cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 255), 2)
    cv2.line(image, (center, 0), (center, image.shape[0]), (0, 255, 0), 3)
    cv2.putText(image, str(center), (center+10, 100), cv2.FONT_HERSHEY_PLAIN, 2, (0, 255, 0), 2)
    cv2.line(image, (center, 0), (center, image.shape[0]), (0, 255, 0), 3)
    
    return image, cx, center
  
    
# 차량 이동 함수 
def find_way(vanishing, center):
    diff = vanishing - center
    #print(diff)
    if diff < -20:
        left()
    elif diff > 20:
        right()
    else:
        forward()

# Initialize MazeRunner, gets MODI class
# Add needed modules
def init_MR(bundle):
    print('modules list\n', bundle.modules)
    motor = bundle.motors[0]
    ir_side = bundle.irs[1]
    ir_front = bundle.irs[0]
    return len(bundle.modules), motor , ir_side, ir_front

# Checks module connection status by comparing module numbers.
def is_connected(curr_num):
    if curr_num != module_num:
        print('\n--------interrupt!!!---------')
        print('Some modules disconnected!!')
        return False
    else:
        return True

    
# MODI 모터 제어 함수 

# 주차공간 진입시 사용하는 직진함수 
def forward(delay=10, speed=50):
    motor.speed= 0, 0
    time.sleep(0.001)
    # if button.clicked() == True:
    #print('-----forward!!-----')
    for _ in range(delay):
    # mazeprint(ir.distance())
        time.sleep(0.001)
        motor.speed= speed, -speed
        time.sleep(0.001)
    motor.speed =0, 0

# MODI turns left, gets delay arg.
def left(delay=1):
    motor.speed= 0,0
    time.sleep(0.001)
    #print('-----left!!-----')
    for _ in range(delay):
        time.sleep(0.001)
        motor.speed= 30, 30
        time.sleep(0.001)
    motor.speed =0, 0

# MODI turns right, gets delay arg.
def right(delay=1):
    motor.speed = 0, 0
    time.sleep(0.001)
    #print('-----right!!-----')
    for _ in range(delay):
        time.sleep(0.001)
        motor.speed= -30, -30
        time.sleep(0.001)
    motor.speed = 0, 0

    
    
# 주차 관련 함수

# 빈 주차공간 확인 후 90도 턴 하는 동작을 위한 함수 
# 위치 보정을 위해 살짝 앞에 전진 후 90 도 회전 합니다. (보정 필요)
def right90(delay=13):
    motor.speed = 0, 0
    time.sleep(0.001)
    
    motor.speed= 30, -30
    time.sleep(1)
    motor.speed =0, 0
    
    for _ in range(delay):
        time.sleep(0.001)
        motor.speed= 0, -80
        time.sleep(0.001)
    motor.speed = 0, 0
# 90도 턴 후 라인에 안정적으로 진입을 위한 후진 동작 함수 입니다.    
def backward(delay=5, speed=100):
    motor.speed= 0, 0
    time.sleep(0.001)
   
    for _ in range(delay):
        time.sleep(0.001)
        motor.speed= -speed, speed
        time.sleep(0.001)
    motor.speed =0, 0
# 주차 공간 찾기 위한 사이드 ir 센서 검사 함수
def check_side_dis(sensor_val):
    if sensor_val < 5 and sensor_val!= 0:
        return False
    else:
        return True
# 주차 진입 시 전방 ir센서 검사 함수     
def check_frt_dis(sensor_val):
    if sensor_val > 70:
        return False
    else:
        return True
# 빈 주차공간 탐색시 사용하는 직진 함수
def park_forward(delay=5, speed=50):
    motor.speed= 0, 0
    time.sleep(0.001)
    # if button.clicked() == True:
    #print('-----forward!!-----')
    for _ in range(delay):
    # mazeprint(ir.distance())
        time.sleep(0.001)
        motor.speed= speed, -speed
        time.sleep(0.001)
    motor.speed =0, 0
    
# 주차 경보 관련 함수    
    
def dis_checker_led(sensor_val):
    if sensor_val > 15 and sensor_val!= 0:
        led.red = 255
        time.sleep(1)
        print( )
    if sensor_val > 5 and sensor_val <=15 and sensor_val !=0:
        led.rgb = 255, 255, 0
        time.sleep(1)
    if sensor_val <=5 :
        led.green = 255
        time.sleep(1)
        
        
def dis_checker_speaker(sensor_val):
    if sensor_val > 15 and sensor_val!= 0:
        speak.frequency = 6000
        speak.volume = 100
        time.sleep(1)
        speak.turn_off()
        time.sleep(1)
    if sensor_val > 10 and sensor_val <=15 and sensor_val !=0:
        speak.frequency = 3000
        speak.volume = 70
        time.sleep(1)
        speak.turn_off()
        time.sleep(1)
    if sensor_val > 5 and sensor_val <=10 and sensor_val !=0:
        speak.frequency = 2000
        speak.volume = 50
        time.sleep(1)
        speak.turn_off()
        time.sleep(1)

○ 모듈 객체화  
 모터, 측면 ir센서, 정면 ir센서, LED 모듈, 스피커 모듈 활성화 

In [3]:
module_num, motor,ir_side, ir_front = init_MR(bundle)
led = bundle.leds[0]
speak = bundle.speakers[0]
speak.tune = 800, 70

time.sleep(1)
print('MODI Connected!')
print('motor 연결',motor)
print('ir_side 연결',ir_side)
print('ir_front 연결',ir_front)

modules list
 [<modi.module.output_module.motor.Motor object at 0x9f8f6c90>, <modi.module.setup_module.battery.Battery object at 0x9f90cd90>, <modi.module.output_module.led.Led object at 0xa032adb0>, <modi.module.input_module.ir.Ir object at 0x9f8f6bb0>, <modi.module.output_module.speaker.Speaker object at 0xa033a210>, <modi.module.input_module.ir.Ir object at 0x9f90c870>]
MODI Connected!
motor 연결 Motor (2606)
ir_side 연결 Ir (1862)
ir_front 연결 Ir (620)
 (1061) has been disconnected!!


○ 카메라 모듈 세팅

In [4]:
cap = AICamera()
print('카메라 연결',cap.is_opened())

cap.set_window()
print('set window')

카메라 연결 True


'Window'

'Frame Rate'

set window
 (1061) has been disconnected!!


○ 주차 공간 진입 파트 

In [5]:
empty_check = True
front_check = True

while front_check:
        time.sleep(0.01)
        frame = cap.read()
        frame = cv2.flip(frame,1)
        
        
        canny_image = canny(frame)
        cropped_image = region_of_interest(canny_image)


        # 선분 검출 함수         
        lines = cv2.HoughLinesP(cropped_image, 1 , np.pi/180, 100, np.array([]), minLineLength=10, maxLineGap=1)      

        if str(type(lines)) == "<class 'NoneType'>":
            lines = [[[0, 0 ,0 ,0]]]
        elif len(lines) < 2:
            continue

        # 선분이 2개 검출될때 실행 
        if len(lines) >= 2 :
            averaged_lines = average_slope_intercept(frame, lines)
            find_vanishing(frame, averaged_lines)
            line_image = t_display_lines(frame, averaged_lines)
            vanishing_line, vanishing, center = find_vanishing(line_image, averaged_lines)
            combo_image = cv2.addWeighted(frame, 0.8, vanishing_line, 1, 1)
            find_way(vanishing, center)
            
        front_check = check_frt_dis(ir_front.proximity)
        # 주차 거리에 따른 경보 시스템 작동 함수
        dis_checker_led(ir_front.proximity)
        dis_checker_speaker(ir_front.proximity)
        
# 영상 출력 함수
#         cap.show(frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
            
motor.speed = 0, 0
time.sleep(0.001)
print('finish')
cv2.destroyAllWindows()


 (620) has been disconnected!!


KeyboardInterrupt: 